In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers, optimizers, Sequential, metrics

In [3]:
(x, y), (x_val, y_val) = keras.datasets.cifar10.load_data()
batchsz=128
print('y: ', y.shape, 'x:', x.shape)
y = tf.squeeze(y) # 将y的多余维度1去除， 删除size为1的维度
y_val = tf.squeeze(y_val)
y = tf.one_hot(y, depth=10) # [50k, 10]
y_val = tf.one_hot(y_val, depth=10) # [10k, 10]
print('datasets:', x.shape, y.shape, x_val.shape, y_val.shape, x.min(), x.max())

y:  (50000, 1) x: (50000, 32, 32, 3)
datasets: (50000, 32, 32, 3) (50000, 10) (10000, 32, 32, 3) (10000, 10) 0 255


In [6]:
def preprocess(x, y):
    # [0~255] => [-1~1]
    x = 2 * tf.cast(x, dtype=tf.float32) / 255. - 1.
    y = tf.cast(y, dtype=tf.int32)
    return x,y

train_db = tf.data.Dataset.from_tensor_slices((x,y))
train_db = train_db.map(preprocess).shuffle(10000).batch(batchsz)
test_db = tf.data.Dataset.from_tensor_slices((x_val, y_val))
test_db = test_db.map(preprocess).batch(batchsz)
sample = next(iter(train_db))
print('batch:', sample[0].shape, sample[1].shape)

batch: (128, 32, 32, 3) (128, 10)


In [7]:
class MyDense(layers.Layer):
    # to replace standard layers.Dense()
    def __init__(self, inp_dim, outp_dim):
        super(MyDense, self).__init__()

        self.kernel = self.add_weight('w', [inp_dim, outp_dim])
        # self.bias = self.add_variable('b', [outp_dim])

    def call(self, inputs, training=None):

        x = inputs @ self.kernel
        return x

class MyNetwork(keras.Model):

    def __init__(self):
        super(MyNetwork, self).__init__()

        self.fc1 = MyDense(32*32*3, 256)
        self.fc2 = MyDense(256, 128)
        self.fc3 = MyDense(128, 64)
        self.fc4 = MyDense(64, 32)
        self.fc5 = MyDense(32, 10)



    def call(self, inputs, training=None):
        """

        :param inputs: [b, 32, 32, 3]
        :param training:
        :return:
        """
        x = tf.reshape(inputs, [-1, 32*32*3])
        # [b, 32*32*3] => [b, 256]
        x = self.fc1(x)
        x = tf.nn.relu(x)
        # [b, 256] => [b, 128]
        x = self.fc2(x)
        x = tf.nn.relu(x)
        # [b, 128] => [b, 64]
        x = self.fc3(x)
        x = tf.nn.relu(x)
        # [b, 64] => [b, 32]
        x = self.fc4(x)
        x = tf.nn.relu(x)
        # [b, 32] => [b, 10]
        x = self.fc5(x)

        return x

In [9]:
network = MyNetwork()
network.compile(optimizer=optimizers.Adam(lr=1e-3),
                loss=tf.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
network.fit(train_db, epochs=15, validation_data=test_db, validation_freq=1)

network.evaluate(test_db)
weights_path = 'weights/cifar/weights.ckpt'
network.save_weights(weights_path)
del network
print('saved to ckpt/weights.ckpt')


network = MyNetwork()
network.compile(optimizer=optimizers.Adam(learning_rate=1e-3),
                loss=tf.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
network.load_weights(weights_path)
print('loaded weights from file.')
network.evaluate(test_db)

Epoch 1/15


C:\Users\yawei\AppData\Local\Temp\ipykernel_29812\3516603753.py:6: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  self.kernel = self.add_variable('w', [inp_dim, outp_dim])
f:\Code\python\AILearn\env\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


391/391 [==============================] - 5s 11ms/step - loss: 1.7152 - accuracy: 0.3954 - val_loss: 1.5507 - val_accuracy: 0.4539
Epoch 2/15
391/391 [==============================] - 4s 11ms/step - loss: 1.4955 - accuracy: 0.4743 - val_loss: 1.4976 - val_accuracy: 0.4764
Epoch 3/15
391/391 [==============================] - 4s 11ms/step - loss: 1.3965 - accuracy: 0.5091 - val_loss: 1.4379 - val_accuracy: 0.4934
Epoch 4/15
391/391 [==============================] - 4s 11ms/step - loss: 1.3159 - accuracy: 0.5373 - val_loss: 1.4176 - val_accuracy: 0.5083
Epoch 5/15
391/391 [==============================] - 4s 11ms/step - loss: 1.2439 - accuracy: 0.5630 - val_loss: 1.4040 - val_accuracy: 0.5172
Epoch 6/15
391/391 [==============================] - 5s 12ms/step - loss: 1.1812 - accuracy: 0.5832 - val_loss: 1.3978 - val_accuracy: 0.5192
Epoch 7/15
391/391 [==============================] - 4s 11ms/step - loss: 1.1239 - accuracy: 0.6041 - val_loss: 1.4164 - val_accuracy: 0.5179
Epoch 8/15

[1.6974825859069824, 0.5167999863624573]

## cifar100 - VGG


### VGG

![VGG](../images/vgg.png)

In [1]:
import  tensorflow as tf
from tensorflow import keras
from keras import layers, optimizers, datasets, Sequential
tf.random.set_seed(2345)

In [2]:
(x,y), (x_test, y_test) = datasets.cifar100.load_data()

In [3]:
def preprocess(x, y):
    # [0~1]
    x = tf.cast(x, dtype=tf.float32) / 255.
    y = tf.cast(y, dtype=tf.int32)
    return x,y

y = tf.squeeze(y, axis=1)
y_test = tf.squeeze(y_test, axis=1)
print(x.shape, y.shape, x_test.shape, y_test.shape)


train_db = tf.data.Dataset.from_tensor_slices((x,y))
train_db = train_db.shuffle(1000).map(preprocess).batch(128)

test_db = tf.data.Dataset.from_tensor_slices((x_test,y_test))
test_db = test_db.map(preprocess).batch(64)

sample = next(iter(train_db))
print('sample:', sample[0].shape, sample[1].shape,
      tf.reduce_min(sample[0]), tf.reduce_max(sample[0]))

(50000, 32, 32, 3) (50000,) (10000, 32, 32, 3) (10000,)
sample: (128, 32, 32, 3) (128,) tf.Tensor(0.0, shape=(), dtype=float32) tf.Tensor(1.0, shape=(), dtype=float32)


In [9]:
conv_layers = [ # 5 units of conv + max pooling

    # 注意这里64， 128， 256都是kernal channel，不是数据的维度。
    # 一般来讲，channel慢慢变大，h, w会慢慢缩小，但图片所包含的信息会逐步变大，从低维信息变为高维信息
    # channel的增加一般是成倍增加，但不是必须
    # unit 1
    layers.Conv2D(64, kernel_size=[3, 3], padding="same", activation=tf.nn.relu),
    layers.Conv2D(64, kernel_size=[3, 3], padding="same", activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2, 2], strides=2, padding='same'),

    # unit 2
    layers.Conv2D(128, kernel_size=[3, 3], padding="same", activation=tf.nn.relu),
    layers.Conv2D(128, kernel_size=[3, 3], padding="same", activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2, 2], strides=2, padding='same'),

    # unit 3
    layers.Conv2D(256, kernel_size=[3, 3], padding="same", activation=tf.nn.relu),
    layers.Conv2D(256, kernel_size=[3, 3], padding="same", activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2, 2], strides=2, padding='same'),

    # unit 4
    layers.Conv2D(512, kernel_size=[3, 3], padding="same", activation=tf.nn.relu),
    layers.Conv2D(512, kernel_size=[3, 3], padding="same", activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2, 2], strides=2, padding='same'),

    # unit 5
    layers.Conv2D(512, kernel_size=[3, 3], padding="same", activation=tf.nn.relu),
    layers.Conv2D(512, kernel_size=[3, 3], padding="same", activation=tf.nn.relu),
    layers.MaxPool2D(pool_size=[2, 2], strides=2, padding='same')

]

conv_net = Sequential(conv_layers)

fc_net = Sequential([
    layers.Dense(256, activation=tf.nn.relu),
    layers.Dense(128, activation=tf.nn.relu),
    layers.Dense(100, activation=None),
])

conv_net.build(input_shape=[None, 32, 32, 3])
fc_net.build(input_shape=[None, 512])
optimizer = optimizers.Adam(learning_rate=1e-4)

# [1, 2] + [3, 4] => [1, 2, 3, 4]
# variables在外定义即可，在gradient里，通过apply_gradients不断更新本体
variables = conv_net.trainable_variables + fc_net.trainable_variables


variables [<tf.Variable 'conv2d_10/kernel:0' shape=(3, 3, 3, 64) dtype=float32, numpy=
array([[[[-0.080576  ,  0.05495319,  0.05049922, ..., -0.01842894,
          -0.05975224,  0.00953653],
         [ 0.02608326,  0.06499049, -0.08049025, ..., -0.05459891,
           0.0335414 , -0.0614499 ],
         [-0.07182931, -0.07748628,  0.03153197, ..., -0.05410354,
          -0.05424809, -0.09008971]],

        [[-0.08758618, -0.04248689, -0.02685919, ..., -0.00991104,
           0.00658701,  0.08466525],
         [-0.09593751, -0.05741454,  0.06089693, ..., -0.09862255,
          -0.08192685,  0.01197711],
         [-0.00183346,  0.00316307, -0.06980264, ..., -0.07046674,
           0.0290796 ,  0.06540215]],

        [[-0.00100776,  0.04332659,  0.0056214 , ...,  0.09439854,
           0.01200542, -0.01802916],
         [-0.07453839,  0.04363552,  0.03259254, ..., -0.01680037,
          -0.04930074, -0.06457216],
         [ 0.03973146,  0.01033596, -0.02545224, ...,  0.07445037,
          

In [11]:
for epoch in range(10):

    for step, (x,y) in enumerate(train_db):

        with tf.GradientTape() as tape:
            # [b, 32, 32, 3] => [b, 1, 1, 512]
            out = conv_net(x)
            # flatten, => [b, 512]
            out = tf.reshape(out, [-1, 512])
            # [b, 512] => [b, 100]
            logits = fc_net(out)
            # [b] => [b, 100]
            y_onehot = tf.one_hot(y, depth=100)
            # compute loss
            loss = tf.losses.categorical_crossentropy(y_onehot, logits, from_logits=True)
            loss = tf.reduce_mean(loss)

        grads = tape.gradient(loss, variables)
        optimizer.apply_gradients(zip(grads, variables))

        if step % 100 == 0:
            print(epoch, step, 'loss:', float(loss))



    total_num = 0
    total_correct = 0
    for x,y in test_db:

        out = conv_net(x)
        out = tf.reshape(out, [-1, 512])
        logits = fc_net(out)
        prob = tf.nn.softmax(logits, axis=1)# 转为onehot, 并将大值放大， 小值缩小，拉大对比
        pred = tf.argmax(prob, axis=1)
        pred = tf.cast(pred, dtype=tf.int32)

        correct = tf.cast(tf.equal(pred, y), dtype=tf.int32)
        correct = tf.reduce_sum(correct)

        total_num += x.shape[0]
        total_correct += int(correct)

    acc = total_correct / total_num
    print(epoch, 'acc:', acc)

0 0 loss: 4.601833343505859


KeyboardInterrupt: 